In [1]:
from transformers import BertTokenizer, BertModel

2024-03-31 13:03:18.334331: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2024-03-31 13:03:18.334393: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tokenizer = "/slipstream_old/home/juliusherzog/latinbert/models/subword_tokenizer_latin/latin.subword.encoder"
                    
# Load pre-trained model (weights)
model = "/slipstream_old/home/juliusherzog/latinbert/models/latin_bert/"

In [11]:
import argparse, sys
from cltk.tokenizers.lat.lat import LatinWordTokenizer as WordTokenizer
from cltk.tokenizers.lat.lat import LatinPunktSentenceTokenizer as SentenceTokenizer
from tensor2tensor.data_generators import text_encoder
import numpy as np
import torch
from torch import nn
from transformers import BertModel, BertPreTrainedModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LatinBERT():

	def __init__(self, tokenizerPath=None, bertPath=None):
		encoder = text_encoder.SubwordTextEncoder(tokenizerPath)
		self.wp_tokenizer = LatinTokenizer(encoder)
		self.model = BertLatin(bertPath=bertPath)
		self.model.to(device)

	def get_batches(self, sentences, max_batch, tokenizer):

			maxLen=0
			for sentence in sentences:
				length=0
				for word in sentence:
					toks=tokenizer.tokenize(word)
					length+=len(toks)

				if length> maxLen:
					maxLen=length

			all_data=[]
			all_masks=[]
			all_labels=[]
			all_transforms=[]

			for sentence in sentences:
				tok_ids=[]
				input_mask=[]
				labels=[]
				transform=[]

				all_toks=[]
				n=0
				for idx, word in enumerate(sentence):
					toks=tokenizer.tokenize(word)
					all_toks.append(toks)
					n+=len(toks)

				cur=0
				for idx, word in enumerate(sentence):
					toks=all_toks[idx]
					ind=list(np.zeros(n))
					for j in range(cur,cur+len(toks)):
						ind[j]=1./len(toks)
					cur+=len(toks)
					transform.append(ind)

					tok_ids.extend(tokenizer.convert_tokens_to_ids(toks))

					input_mask.extend(np.ones(len(toks)))
					labels.append(1)

				all_data.append(tok_ids)
				all_masks.append(input_mask)
				all_labels.append(labels)
				all_transforms.append(transform)

			lengths = np.array([len(l) for l in all_data])

			# Note sequence must be ordered from shortest to longest so current_batch will work
			ordering = np.argsort(lengths)
			
			ordered_data = [None for i in range(len(all_data))]
			ordered_masks = [None for i in range(len(all_data))]
			ordered_labels = [None for i in range(len(all_data))]
			ordered_transforms = [None for i in range(len(all_data))]
			

			for i, ind in enumerate(ordering):
				ordered_data[i] = all_data[ind]
				ordered_masks[i] = all_masks[ind]
				ordered_labels[i] = all_labels[ind]
				ordered_transforms[i] = all_transforms[ind]

			batched_data=[]
			batched_mask=[]
			batched_labels=[]
			batched_transforms=[]

			i=0
			current_batch=max_batch

			while i < len(ordered_data):

				batch_data=ordered_data[i:i+current_batch]
				batch_mask=ordered_masks[i:i+current_batch]
				batch_labels=ordered_labels[i:i+current_batch]
				batch_transforms=ordered_transforms[i:i+current_batch]

				max_len = max([len(sent) for sent in batch_data])
				max_label = max([len(label) for label in batch_labels])

				for j in range(len(batch_data)):
					
					blen=len(batch_data[j])
					blab=len(batch_labels[j])

					for k in range(blen, max_len):
						batch_data[j].append(0)
						batch_mask[j].append(0)
						for z in range(len(batch_transforms[j])):
							batch_transforms[j][z].append(0)

					for k in range(blab, max_label):
						batch_labels[j].append(-100)

					for k in range(len(batch_transforms[j]), max_label):
						batch_transforms[j].append(np.zeros(max_len))

				batched_data.append(torch.LongTensor(batch_data))
				batched_mask.append(torch.FloatTensor(batch_mask))
				batched_labels.append(torch.LongTensor(batch_labels))
				batched_transforms.append(torch.FloatTensor(batch_transforms))

				bsize=torch.FloatTensor(batch_transforms).shape
				
				i+=current_batch

				# adjust batch size; sentences are ordered from shortest to longest so decrease as they get longer
				if max_len > 100:
					current_batch=12
				if max_len > 200:
					current_batch=6

			return batched_data, batched_mask, batched_transforms, ordering


	def get_berts(self, raw_sents):
		sents = convert_to_toks(raw_sents)
		batch_size = 32
		batched_data, batched_mask, batched_transforms, ordering = self.get_batches(sents, batch_size, self.wp_tokenizer)

		ordered_preds = []
		# Corrected handling of outputs and attentions
		for b in range(len(batched_data)):
			size = batched_transforms[b].shape
			b_size = size[0]
			out, attentions = self.model.forward(batched_data[b], attention_mask=batched_mask[b], transforms=batched_transforms[b])  # This now correctly captures the tuple's items
			out = out.detach()  # Detach only the output tensor
			out = out.cpu()
			for row in range(b_size):
				ordered_preds.append([np.array(r) for r in out[row]])

		preds_in_order = [None for i in range(len(sents))]

		for i, ind in enumerate(ordering):
			preds_in_order[ind] = ordered_preds[i]

		bert_sents = []

		for idx, sentence in enumerate(sents):
			bert_sent = []

			bert_sent.append(("[CLS]", preds_in_order[idx][0]))

			for t_idx in range(1, len(sentence)-1):
				token = sentence[t_idx]
				pred = preds_in_order[idx][t_idx]
				bert_sent.append((token, pred))

			bert_sent.append(("[SEP]", preds_in_order[idx][len(sentence)-1]))

			bert_sents.append(bert_sent)

		return bert_sents, attentions 


class LatinTokenizer():
	def __init__(self, encoder):
		self.vocab={}
		self.reverseVocab={}
		self.encoder=encoder

		self.vocab["[PAD]"]=0
		self.vocab["[UNK]"]=1
		self.vocab["[CLS]"]=2
		self.vocab["[SEP]"]=3
		self.vocab["[MASK]"]=4

		for key in self.encoder._subtoken_string_to_id:
			self.vocab[key]=self.encoder._subtoken_string_to_id[key]+5
			self.reverseVocab[self.encoder._subtoken_string_to_id[key]+5]=key


	def convert_tokens_to_ids(self, tokens):
		wp_tokens=[]
		for token in tokens:
			if token == "[PAD]":
				wp_tokens.append(0)
			elif token == "[UNK]":
				wp_tokens.append(1)
			elif token == "[CLS]":
				wp_tokens.append(2)
			elif token == "[SEP]":
				wp_tokens.append(3)
			elif token == "[MASK]":
				wp_tokens.append(4)

			else:
				wp_tokens.append(self.vocab[token])

		return wp_tokens

	def tokenize(self, text):
		tokens=text.split(" ")
		wp_tokens=[]
		for token in tokens:

			if token in {"[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"}:
				wp_tokens.append(token)
			else:

				wp_toks=self.encoder.encode(token)

				for wp in wp_toks:
					wp_tokens.append(self.reverseVocab[wp+5])

		return wp_tokens

def convert_to_toks(sents):

	sent_tokenizer = SentenceTokenizer()
	word_tokenizer = WordTokenizer()

	all_sents=[]

	for data in sents:
		text=data.lower()

		sents=sent_tokenizer.tokenize(text)
		for sent in sents:
			tokens=word_tokenizer.tokenize(sent)
			filt_toks=[]
			filt_toks.append("[CLS]")
			for tok in tokens:
				if tok != "":
					filt_toks.append(tok)
			filt_toks.append("[SEP]")

			all_sents.append(filt_toks)

	return all_sents




class BertLatin(nn.Module):

	def __init__(self, bertPath=None):
		super(BertLatin, self).__init__()

		self.bert = BertModel.from_pretrained(bertPath)
		self.bert.eval()
		
	def forward(self, input_ids, token_type_ids=None, attention_mask=None, transforms=None):

		input_ids = input_ids.to(device)
		attention_mask = attention_mask.to(device)
		transforms = transforms.to(device)
		
		outputs = self.bert.forward(input_ids, token_type_ids=None, attention_mask=attention_mask)
		# 'outputs' from BERT is a transformers.modeling_outputs.BaseModelOutputWithCrossAttentions object
		sequence_outputs = outputs[0]
		pooled_outputs = outputs[1]
		
		all_layers=sequence_outputs
		out=torch.matmul(transforms,all_layers)
		return out
# python3 scripts/gen_berts.py --bertPath models/latin_bert/ --tokenizerPath models/subword_tokenizer_latin/latin.subword.encoder
# if __name__ == "__main__":
# 
# 	parser = argparse.ArgumentParser()
# 	parser.add_argument('-b', '--bertPath', help='path to pre-trained BERT', required=True)
# 	parser.add_argument('-t', '--tokenizerPath', help='path to Latin WordPiece tokenizer', required=True)
# 	
# 	args = vars(parser.parse_args())
# 
# 	bertPath=args["bertPath"]
# 	tokenizerPath=args["tokenizerPath"]			
# 
# 	bert=LatinBERT(tokenizerPath=tokenizerPath, bertPath=bertPath)
# 
# 	sents=["arma virumque cano", "arma gravi numero violentaque bella parabam"]
# 	
# 	bert_sents=bert.get_berts(sents)
# 
# 	for sent in bert_sents:
# 		for (token, bert) in sent:
# 			print("%s\t%s" % ( token, ' '.join(["%.5f" % x for x in bert])))
# 		print()

	



In [9]:
# Assuming you have an instance `bert` of LatinBERT and a sentence prepared
raw_sents = ["arma virumque cano, Troiae qui primus ab oris"]  # Example sentence
sents = convert_to_toks(raw_sents)  # Convert sentences to token format

# Assuming `get_batches` is appropriately defined to return tensors for input_ids and attention_mask
batched_data, batched_mask, _, _ = bert.get_batches(sents, 1, bert.wp_tokenizer)  # batch_size=1 for simplicity

# Extract output and attentions
out, attentions = bert.model(batched_data[0], attention_mask=batched_mask[0])  # Processing a single batch

# Ensure attentions is a list of tensors with shape [1, num_heads, seq_len, seq_len]


AttributeError: 'NoneType' object has no attribute 'to'

In [3]:
sents = ["Si quid est in me ingeni, iudices, quod sentio quam sit exiguum, aut si qua exercitatio dicendi in qua me non infitior mediocriter esse versatum, aut si huiusce rei ratio aliqua ab optimarum artium studiis ac disciplina profecta, a qua ego nullum confiteor aetatis meae tempus abhorruisse, earum rerum omnium vel in primis hic A. Licinius fructum a me repetere prope suo iure debet.",
         "Ac ne quis a nobis hoc ita dici forte miretur, quod alia quaedam in hoc facultas sit ingeni, neque haec dicendi ratio aut disciplina, ne nos quidem huic uni studio penitus umquam dediti fuimus."]
bert = LatinBERT(tokenizerPath="/slipstream_old/home/juliusherzog/latinbert/models/subword_tokenizer_latin/latin.subword.encoder",
                 bertPath="/slipstream_old/home/juliusherzog/latinbert/models/latin_bert/")
bert_sents, attentions = bert.get_berts(sents)


/tmp/ipykernel_1185417/3480443343.py:124: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  batched_transforms.append(torch.FloatTensor(batch_transforms))


In [6]:
outputs = bert.model(input_ids=sents, attention_mask=attention_mask)
attentions = outputs[-1]  # Assuming attentions are the last item in the outputs tuple

# Now check the shape of the first layer's attentions for the first example in the batch
if attentions is not None and len(attentions) > 0:
    print(attentions[0].shape)  # Expected shape: (batch_size, num_heads, sequence_length, sequence_length)
else:
    print("Attention weights are not available or empty.")

NameError: name 'attention_mask' is not defined

In [4]:
# Example of checking the attention weights shape
if attentions is not None and len(attentions) > 0:
    print(attentions[0].shape)  # Print the shape of the first layer's attentions
else:
    print("Attention weights are not available or empty.")

torch.Size([768])


In [2]:
from transformers import BertModel, BertTokenizer
from bertviz import model_view

model_path = "/slipstream_old/home/juliusherzog/latinbert/models/latin_bert/"
tokenizer_path = "/slipstream_old/home/juliusherzog/latinbert/models/subword_tokenizer_latin/latin.subword.encoder"  

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

# Load the model
model = BertModel.from_pretrained(model_path, output_attentions=True)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Special tokens have been added in the vocabulary, make sure the associated word emebedding are fine-tuned or trained.


In [ ]:
# Example text
text = "Si quid est in me ingeni, iudices, quod sentio quam sit exiguum, aut si qua exercitatio dicendi in qua me non infitior mediocriter esse versatum, aut si huiusce rei ratio aliqua ab optimarum artium studiis ac disciplina profecta, a qua ego nullum confiteor aetatis meae tempus abhorruisse, earum rerum omnium vel in primis hic A. Licinius fructum a me repetere prope suo iure debet."

from transformers import BertTokenizer, BertModel
import torch

In [12]:
tokenizer_path = "/slipstream_old/home/juliusherzog/latinbert/models/subword_tokenizer_latin/latin.subword.encoder"
model_path = "/slipstream_old/home/juliusherzog/latinbert/models/latin_bert/"

# Ensure that these paths are correct and point to the directory containing tokenizer and model files.
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
Special tokens have been added in the vocabulary, make sure the associated word emebedding are fine-tuned or trained.


In [18]:
!pip install sentencepiece

In [14]:
# If using a custom model that isn't directly compatible with BertModel, adjustments might be necessary.
model = BertModel.from_pretrained(model_path, output_attentions=True)

In [20]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.load(tokenizer_path)

RuntimeError: Internal: src/sentencepiece_processor.cc(1101) [model_proto->ParseFromArray(serialized.data(), serialized.size())] 

In [15]:
text = "Your example text here."
print(type(text))  # Should be <class 'str'>
print(text)        # Should print the actual text, ensuring it's not None or empty

<class 'str'>
Your example text here.


In [22]:
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.

In [10]:
print(inputs.keys())
print(inputs['input_ids'].shape)
print(inputs['attention_mask'].shape)
outputs = model(**inputs)
attention = outputs.attentions

NameError: name 'inputs' is not defined